In [24]:
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup
from keybert import KeyBERT

In [25]:
uri = "mongodb://localhost:27017/"
client = MongoClient(uri)

db = client.get_database("cs6675")
web_crawler = db.get_collection("web_crawler")

starting_url = "https://www.cc.gatech.edu/"

# model = KeyBERT('distilbert-base-nli-mean-tokens')

from rake_nltk import Rake
import nltk
 
nltk.download('stopwords')
nltk.download('punkt')

rake_nltk_var = Rake()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emily\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emily\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
visited = set()
q = [starting_url]

while len(q) > 0 and len(visited) < 1000:
    url = q.pop(0)
    if url not in visited:
        print(len(visited))
        try: 
            x = requests.get(url, timeout=5)

            if x.status_code == 200:
                visited.add(url)
                soup = BeautifulSoup(x.text, 'html.parser')
                relative_links = soup.find_all('a')
                text = soup.get_text(' ', strip=True)[:5000]

                doc = {"url": url, "text": text}
                web_crawler.update_one({"url": url}, {"$set": doc}, upsert=True)

                for link in relative_links:
                    next_url = link.get("href")
                    if next_url and "cc.gatech.edu" in next_url and next_url not in visited:
                        q.append(next_url)
        except:
            continue

print(len(visited))
        